## Réseaux Fully Connected (Dense)


In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
keras.__version__

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
im_train = x_train.reshape(60000,28,28)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
plt.imshow((im_train[0]))

In [ ]:
print(y_train[0])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(im_train[8], cmap=plt.cm.binary)


In [ ]:
print(y_train[8])

In [ ]:
import numpy
from numpy import linalg
numpy.set_printoptions(precision=2, suppress=True, linewidth=120)
print(numpy.matrix(im_train[8]))



In [ ]:

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)


print(x_train.shape)
print(x_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
print(y_train[0])

- L’objectif de cette seconde séance de travaux pratiques est de prendre en main la librairie Keras https://keras.io/ pour utiliser et entraîner des réseaux de neurones profonds.

- Avec Keras, les réseaux de neurones avec une structure de chaîne (réseaux feedforward), s’utilisent de la manière suivante:

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:

# model = Sequential()   .... pour TF1
model = tf.keras.models.Sequential()

- On crée ainsi un réseau de neurones vide. On peut alors ajouter des couches avec la fonction add.

##  Régression Logistique avec Keras

Par exemple, l’ajout d’une couche de projection linéaire (couche complètement connectée) de taille 10, suivi de l’ajout d’une couche d’activation de type softmax, peuvent s’effectuer de la manière suivante :

In [ ]:
from tensorflow.keras.layers import Dense, Activation

model.add(tf.keras.layers.Dense(10 , activation='softmax',  input_dim=784 ))


- On peut ensuite visualiser l’architecture du réseau avec la méthode summary() du modèle.

#### Question :
Vérifier le nombre de paramètres du réseau à apprendre dans la méthode summary(). 
Écrire un script permettant de créer le réseau de neurone ci-dessus.


In [ ]:
model.summary()

- Avec Keras, on va compiler le modèle en lui passant :
    - un loss (ici l’entropie croisée) ;
    - une méthode d’optimisation (ici une descente de gradient stochastique, stochastic gradient descent, sgd) ; 
    - une métrique d’évaluation (ici le taux de bonne prédiction des catégories, accuracy).

In [ ]:
#from keras.optimizers import SGD    : TF1
from tensorflow.keras.optimizers import SGD   # TF2

opt = SGD(learning_rate=0.1)

model.compile(loss='categorical_crossentropy', optimizer= opt, metrics=['accuracy'])

Enfin, l’apprentissage du modèle sur des données d’apprentissage est mis en place avec la méthode fit :

In [ ]:

model.fit(x_train, y_train, batch_size=50, epochs=5,verbose=1)

- batch_size correspond au nombre d’exemples utilisé pour estimer le gradient de la fonction de coût.
- epochs est le nombre d’époques (i.e. passages sur l’ensemble des exemples de la base d’apprentissage) lors de la descente de gradient.

- N.B : on rappelle que les étiquettes (labels) données par la supervision doivent être au format one-hot encoding.

On peut ensuite évaluer les performances du modèle sur l’ensemble de test avec la fonction evaluate :

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


Le premier élément de scores renvoie la fonction de coût sur la base de test, le second élément renvoie le taux de bonne détection (accuracy).

### Sauvegader un modèle


 
 On pourra utiliser la méthode suivante pour sauvegarder le modèle appris :

In [ ]:
from keras.models import model_from_yaml
def saveModel(model, savename):
    # serialize model to YAML
    model_yaml = model.to_yaml()
    with open(savename+".yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)
        print("Yaml Model ",savename,".yaml saved to disk")
    # serialize weights to HDF5
    model.save_weights(savename+".h5")
    print("Weights ",savename,".h5 saved to disk")

### Essayer d'autres modèles en changeant les hyper-paramètres

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10, activation='sigmoid', input_dim =(784)))#input_shape(784,)
model.add(tf.keras.layers.Dense(10, activation='softmax'))



# Déclaration du modèle Tensorflow 1.?
#model = Sequential()
#model.add(Dense(10, activation='sigmoid', input_shape=(784,)))
#model.add(Dense(10, activation='softmax'))

# résumé du modèle
model.summary()

In [ ]:
batch_size = 50
num_classes = 10
epochs=10

model.compile(loss='categorical_crossentropy',optimizer='SGD',  metrics=['accuracy'])

model.fit(x_train, y_train,  batch_size=batch_size, epochs=epochs, verbose=1 )

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

### activation function : relu + new hidden layer

In [ ]:
batch_size = 50
num_classes = 10
epochs=10

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, activation='relu', input_dim =(784)))#input_shape(784,)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

test_loss, test_acc = model.evaluate(x_test, y_test, verbose =0)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

## Réseaux de neurones convolutifs avec Keras

On va maintenant étendre le perceptron de l’exercice précédent pour mettre en place un réseau de neurones convolutif profond, Convolutional Neural Networks, ConvNets.

Écrire un script pour mettre en place un ConvNet.

Les réseaux convolutifs manipulent des images multi-dimensionnelles en entrée (tenseurs). On va donc commencer par reformater les données d’entrée afin que chaque exemple soit de taille 28×28×1.

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

Par rapport aux réseaux complètement connectés, les réseaux convolutifs utilisent les briques élémentaires suivantes :

   1. Des couches de convolution, qui transforment un tenseur d’entrée de taille nx×ny×p

en un tenseur de sortie nx′×ny′×nH, où nH est le nombre de filtres choisi.

Par exemple, une couche de convolution pour traiter les images d’entrée de MNIST peut être créée de la manière suivante :

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

conv=Conv2D(32,kernel_size=(5, 5),activation='sigmoid',input_shape=(28, 28, 1),padding='same')

    - 32 : nombre de filtres.
    - (5, 5) : taille spatiale de chaque filtre (masque de convolution).
    - padding='same' correspond à garder la même taille spatiale en sortie de la convolution.
    
    N.B. : on peut directement inclure dans la couche de convolution la non-linéarité en sortie de la convolution, comme illustré ici dans l’exemple avec une fonction d’activation de type sigmoid.

 

  2. Des couches d’agrégation spatiale (pooling), afin de permettre une invariance aux translations locales. Voici par exemple la manière de déclarer une couche de max-pooling:


In [ ]:
pool = MaxPooling2D(pool_size=(2, 2))

    (2, 2) : la taille spatiale sur laquelle l’opération d’agrégation est effectuée.
    N.B. : par défaut, le pooling est effectué avec un décalage de 2 neurones, dans l’exemple précédent on obtient donc des cartes de sorties avec des tailles spatiales divisées par deux par rapport à la taille d’entrée.

### Exercice : 
Compléter le script pour mettre en place un ConvNet à l’architecture suivante, proche du modèle historique LeNet5 [LBD+89] et montré ci-dessous:

        - Une couche de convolution avec 32 filtres de taille 5×5, suivie d’une non linéarité de type sigmoïde  puis d’une couche de max pooling de taille 2×2.
        - Une seconde couche de convolution avec 64 filtres de taille 5×5, suivie d’une non linéarité de type sigmoïde puis d’une couche de max pooling de taille 2×2.
        - Comme dans le réseau LeNet, on considérera la sortie du second bloc convolutif comme un vecteur, ce que revient à « mettre à plat » les couches convolutives précédentes (model.add(Flatten())).
        - Une couche complètement connectée de taille 100, suivie d’une non linéarité de type sigmoïde.
        - Une couche complètement connectée de taille 10, suivie d’une non linéarité de type softmax.

In [ ]:
model = tf.keras.models.Sequential()
model.add(conv)
### le reste est à vous de compléter


In [ ]:
model.summary()


    - Apprendre le modèle et évaluer les performances du réseau sur la base de test. Vous devez obtenir un score de l’ordre de 97% pour ce modèle ConvNet.
    - Quelle est le temps d’une époque avec ce modèle convolutif ?


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

test_loss, test_acc = model.evaluate(x_test, y_test, verbose =0)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)